In [6]:
import sys
# Import NDVM modules to load pickled data
# NOTE: Modify based on your environment
sys.path.append('/home/dosoukup/Dataset/drift-katoda-v2/katoda-tools/ndvm/modules')
sys.path.append('/home/dosoukup/Dataset/drift-katoda-v2/katoda-tools/ndvm')
import os
import pickle
import metric1, metric3, metric2x
import json
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.cm as cm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import pandas as pd

In [7]:
# NOTE: Modify based on your report. By default it is enough to change the id number
REPORT_NAME = "metrics/report-dataset-1762895049"

# General Report 

In [8]:
with open(REPORT_NAME+'.csv', 'r') as file:
    basic = json.load(file)
pprint(basic)

{'Analyzed Samples': {'1.aerial.maps.api.here.com': 1,
                      '1.base.maps.api.here.com': 2,
                      '1.pano.maps.api.here.com': 1,
                      '1.traffic.maps.api.here.com': 1,
                      '11bup83sxdss1xze1i3lpol4-wpengine.netdna-ssl.com': 3,
                      '165llx2rviujt4pxo3powmwm-wpengine.netdna-ssl.com': 1,
                      '1e2c11e86f814838a3aa19e5ab23cb9d.events.ubembed.com': 2,
                      '1e2c11e86f814838a3aa19e5ab23cb9d.js.ubembed.com': 1,
                      '27612f80fafe409b9624936387ae115d.js.ubembed.com': 1,
                      '28zynx7e84y43nfqd3f09vk1-wpengine.netdna-ssl.com': 7,
                      '2s7gjr373w3x22jf92z99mgm5w-wpengine.netdna-ssl.com': 8,
                      '2vbgco2aiwja4cl3tl4664zj-wpengine.netdna-ssl.com': 6,
                      '31vod-adaptive.akamaized.net': 1,
                      '320ji72lla203vnm9039ska9-wpengine.netdna-ssl.com': 1,
                      '35z8e83

# Metric 1

In [ ]:
with open(REPORT_NAME+"-metric1/metric1.obj", "rb") as f:
    metric1 = pickle.load(f)

In [ ]:
metric1.ds_redundancy

In [ ]:
data = metric1.redundancy_metadata

In [ ]:
plt.figure(figsize=(10, 6))

for item in data:
    for name, values in item.items():
        x_values = sorted(values.keys())
        y_values = []
        for x in x_values:
            # Calculate the average of the last values (assuming each list has 5 values)
            avg_y = np.mean([v[0] for v in values[x]])
            y_values.append(avg_y)
        plt.plot(x_values, y_values, label=name)

plt.xlabel("X Axis Values")
plt.ylabel("Average of Last Values")
plt.title("Plot of Data")
plt.legend()
plt.grid(True)
plt.show()

# Metric 2x

In [ ]:
# Load the Pickled Metric Object
try:
    pickle_path = os.path.join(f'{REPORT_NAME}-metric2', 'metric2x.obj')
    with open(pickle_path, 'rb') as f:
        metric2_obj = pickle.load(f)
    print("✅ Successfully loaded pickled metric2 object!")
    print(f"   - Classifiers used: {list(metric2_obj.classifiers.keys())}")
    print(f"   - Permutation percentages: {metric2_obj.permutation_percentages}")
    
except FileNotFoundError:
    print(f"❌ Error: Could not find the pickle file at {pickle_path}")
except Exception as e:
    print(f"❌ An error occurred while loading the pickle file: {e}")

In [ ]:
final_scores = metric2_obj.get_score()

In [ ]:
# Display P-Values and Final Scores

print("="*30)
print("      P-Value Table")
print("="*30)

# Create a clean pandas DataFrame for the p-values
p_value_df = pd.DataFrame(
    data=metric2_obj.p_values,
    index=metric2_obj.classifiers.keys(),
    columns=metric2_obj.permutation_percentages
)
print(p_value_df)

print("\n" + "="*30)
print("      Final Score Summary")
print("="*30)

# The final results are already calculated and stored in the object
final_scores = metric2_obj.get_score()
for key, value in final_scores.items():
    if key != "P-value table":
        print(f"{key+':':<20} {value}")

In [ ]:
# Visualize Individual Permutations with Polyfit Trend Line 

print("Generating plot for individual permutation runs with polyfit trend line...")

# Extract data from the loaded object
classifier_names = list(metric2_obj.classifiers.keys())
num_classifiers = len(classifier_names)
initial_scores = metric2_obj.initial_scores[0] # This is a 1D array of the initial scores

# Prepare the plot
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(12, 8))
colors = cm.viridis(np.linspace(0, 1, num_classifiers))

# Get the mean correlations (X-axis for the polyfit)
mean_correlations = metric2_obj.correlations.mean(axis=0)

# --- Plotting each individual permutation run as a scatter point ---
for i, (name, color) in enumerate(zip(classifier_names, colors)):
    # Flatten the arrays to get a list of all correlation scores and corresponding F1-scores
    all_correlations = metric2_obj.correlations.flatten()
    all_scores_for_clf = metric2_obj.permuted_scores[:, :, i].flatten()
    
    # Use a low alpha to see point density
    ax.scatter(all_correlations, all_scores_for_clf, color=color, alpha=0.15, s=20, label=f'{name} (Individual Runs)')

# --- Overlay the polyfit trend lines and the initial score line ---
for i, (name, color) in enumerate(zip(classifier_names, colors)):
    
    # Get the mean scores for this classifier (Y-axis for the polyfit)
    mean_scores = metric2_obj.permuted_scores[:, :, i].mean(axis=0)
    
    # Calculate the polyfit line (degree 1) exactly as the module does
    slope, intercept = np.polyfit(mean_correlations, mean_scores, 1)
    
    # Create X-values for plotting the line (from lowest mean correlation to 1.0)
    x_line = np.array([np.min(mean_correlations), 1.0])
    
    # Calculate the corresponding Y-values for the line
    y_line = slope * x_line + intercept
    
    # Plot the polyfit trend line
    ax.plot(x_line, y_line, linestyle='-', color=color, linewidth=2.5, label=f'{name} (Trend Line)')
    # -----------------------------------------------------------------
    
    # --- Draw the horizontal line for the initial, non-permuted score ---
    initial_score = initial_scores[i]
    ax.axhline(y=initial_score, color=color, linestyle='--', linewidth=2, label=f'{name} (Initial Score)')
    # -------------------------------------------------------------------

# --- Formatting and Finalizing the Plot ---

# Create a clean legend
handles, labels = ax.get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))
ax.legend(unique_labels.values(), unique_labels.keys(), title='Classifier', bbox_to_anchor=(1.04, 1), loc='upper left')

ax.set_title('Distribution of Permutation Results with Trend Line', fontsize=16)
ax.set_xlabel('Correlation with Original Labels', fontsize=12)
ax.set_ylabel('F1-Score (Macro)', fontsize=12)
ax.set_ylim(bottom=0)
ax.set_xlim(left=min(metric2_obj.correlations.flatten()) - 0.05, right=1.05)

# Adjust layout to make room for the legend
plt.tight_layout(rect=[0, 0, 0.85, 1]) # rect=[left, bottom, right, top]
plt.show()

# Metric 2 - legacy

In [ ]:
# This is legacy visualization for the original version of Metric 2 which is now beeing obsoleted

In [ ]:
with open(REPORT_NAME+"-metric2/metric2.obj", "rb") as f:
    metric2 = pickle.load(f)

In [ ]:
metric2.raw_slope

In [ ]:
metric2.print_results()

In [ ]:
colors = cm.rainbow(np.linspace(0, 1, metric2.a[1]))

In [ ]:
pvalues = np.zeros((metric2.a[1],len(metric2.perc)))
colors = cm.rainbow(np.linspace(0, 1, metric2.a[1]))

# plot true values as diamonds
for i, c in zip(range(metric2.a[1]),colors):
    plt.scatter(1.1+i*0.01, metric2.ev.scores.mean(axis=2)[:, i, 0], s=100, color=c, marker='d')
    
plt.legend((metric2.clfs))

# plot lines for true values
for i, c in zip(range(metric2.a[1]),colors):
    plt.plot([0, 1.1+i*0.01], [metric2.ev.scores.mean(axis=2)[:, i, 0], metric2.ev.scores.mean(axis=2)[:, i, 0]], c=c, linestyle='dashed', alpha=0.5)

# plot permutations
colors = cm.rainbow(np.linspace(0, 1, metric2.a[1]))
for j in range(len(metric2.perc)):
    for i, c in zip(range(metric2.a[1]),colors):
        ind = np.where(metric2.perm[:,j,i]<metric2.ev.scores.mean(axis=2)[:, i, 0])
        plt.scatter((metric2.corr[ind,j]), metric2.perm[ind,j,i], color="none", edgecolor=c, alpha=0.3)
        
for j in range(len(metric2.perc)):
    for i, c in zip(range(metric2.a[1]),colors):
        ind = np.where(metric2.perm[:,j,i]>=metric2.ev.scores.mean(axis=2)[:, i, 0])
        plt.scatter((metric2.corr[ind,j]), metric2.perm[ind,j,i], color=c, edgecolor="black", alpha=1)
        pvalues[i,j] = ((len(ind[0])+1)*1.0)/(metric2.nperm+1);

plt.ylabel('Performance (F1)', size=12)
plt.xlabel('Permutation Correlation', size=12)       

plt.plot([0, 1.1], [metric2.perm.min(), metric2.perm.min()], color='red', linestyle='dashed', alpha=0.5)

plt.axis([-0.05, 1.2, 0, 1.1])

plt.show()

pv = pd.DataFrame(data=pvalues, index=metric2.clfs, columns=metric2.perc)

def significant(v):
    return "font-weight: bold; color: red" if v > 0.01 else None

pv.style.applymap(significant)

In [ ]:
names = metric2.clfs
cor = []
per = []
slopes = []
auc_scores = []

for i, c in zip(range(metric2.a[1]),colors):
    for j in range(len(metric2.perc)):
        plt.scatter(np.mean(metric2.corr[:,j]), np.mean(metric2.perm[:,j,i]), color=c, alpha=1)
    
    cor = np.mean(metric2.corr[:,:], axis=0)
    per = np.mean(metric2.perm[:,:,i], axis=0)
    
    auc_score = auc(cor,per)
    slope, intercept = np.polyfit(cor, per, 1)
    plt.plot(cor, slope*cor + intercept, color=c, linewidth=0.8)
    print(names[i], '=', slope)
    slopes = np.append(slopes, slope)
    auc_scores = np.append(auc_scores, auc_score)

plt.legend(names, prop={'size': 8})

maxind = np.argmax(abs(slopes))
maxind_auc = np.argmax(abs(auc_scores))

print('Slope:', np.max(abs(slopes)), '-', names[maxind])
print('AUC:', np.max(abs(auc_scores)), '-', names[maxind_auc])

# Metric 3

In [ ]:
with open(REPORT_NAME+"-metric3/metric3.obj", "rb") as f:
    metric3 = pickle.load(f)

In [ ]:
# The base key
next(iter(basic["Similarity"]["detail"]))

In [ ]:
# All keys
basic["Similarity"]["detail"].keys()

In [ ]:
# All metric 3 metadata
basic["Similarity"]["detail"]